This section explains how to build a basic research assistant using LangChain by combining an LLM with external tools to gather information and respond to user queries.

In [1]:
import sys
import os


sys.path.insert(0, os.path.abspath(".."))

from config import set_environment

set_environment()

In [2]:
from langchain.agents import (
    AgentExecutor, load_tools, create_react_agent
)
from langchain.chat_models import ChatOpenAI
def load_agent() -> AgentExecutor:
    llm = ChatOpenAI(temperature=0, streaming=True)
    tools = load_tools(
        tool_names=["ddg-search", "arxiv", "wikipedia"],
        llm=llm
    )
    return AgentExecutor(
        agent=create_react_agent(llm=llm, tools=tools), tools=tools
    )


The `load_agent()` function creates a LangChain agent by initializing a `ChatOpenAI` LLM with streaming enabled, enhancing user interaction. The `load_tools()` function loads specified tools like DuckDuckGoSearch, arXiv, and Wikipedia, which the agent uses to gather information. The agent combines the LLM with these tools, creating a powerful system for answering queries. The Zero-Shot agent (or ReAct agent) is a versatile agent designed for general-purpose action, capable of dynamically selecting and using the appropriate tools to respond to a wide range of inputs.

The `streaming` parameter in the `ChatOpenAI` constructor is set to `True` to improve the user experience. This enables the response text to be updated incrementally as it is generated, rather than waiting for the entire response to be completed. Currently, only the `OpenAI`, `ChatOpenAI`, and `ChatAnthropic` implementations support this streaming feature.

## Visial Interface

After building an agent with LangChain, the next logical step is to deploy it within an easy-to-use application. Streamlit serves as an excellent framework for this purpose. It is an open-source platform designed specifically for machine learning workflows, making it straightforward to package and present our agent as an interactive web application. With Streamlit, we can quickly create a user-friendly interface for our agent that allows seamless interaction.

### Alternative Frameworks to Streamlit:
Though Streamlit is a great choice, there are several other platforms worth considering, each with its strengths:
- **Gradio**: Similar to Streamlit but optimized for machine learning demos.
- **Dash**: Offers more customization options, though it's more complex to use.
- **Panel**: Focused on building flexible dashboards and web applications.
- **Anvil**: Enables full-stack web app development using Python.
- **Voilà**: Converts Jupyter Notebooks into interactive web applications.
- **Taipy**: Known for its performance and scalability in high-demand applications.

For this example, we’ll stick with **Streamlit**, given its ease of use, broad adoption in the ML community, and simplicity for quickly deploying models and agents as web applications.

In [ ]:
import streamlit as st
from langchain_community.callbacks.streamlit import (
    StreamlitCallbackHandler,
)
chain = load_agent()
st_callback = StreamlitCallbackHandler(st.container())
if prompt := st.chat_input():
    st.chat_message("user").write(prompt)
    with st.chat_message("assistant"):
        st_callback = StreamlitCallbackHandler(st.container())
        response = chain.run(prompt, callbacks=[st_callback])
        st.write(response)


**Imports**  
```python
import streamlit as st
from langchain_community.callbacks.streamlit import StreamlitCallbackHandler
```
- `streamlit as st`: Imports Streamlit, which is used to build the web interface.
- `StreamlitCallbackHandler`: This handler is imported from LangChain’s community tools to enable Streamlit-specific callbacks when using LangChain.

**Load Agent**  
```python
chain = load_agent()
```
- `load_agent()`: This function presumably loads an agent or a chain of tools that have been preconfigured, likely an LLM (like OpenAI) agent or other tools in LangChain. This chain will process the input and generate responses.

**Streamlit Callback Handler**  
```python
st_callback = StreamlitCallbackHandler(st.container())
```
- `StreamlitCallbackHandler(st.container())`: The callback handler is initialized with a Streamlit container, allowing it to send updates to Streamlit's interface during the agent's execution. The `st.container()` function is used to organize the layout of elements in the app.

**Handle User Input**  
```python
if prompt := st.chat_input():
    st.chat_message("user").write(prompt)
```
- `if prompt := st.chat_input()`: This line checks if the user has entered a message in the chat input field. If there is input (`prompt`), the code executes the block inside the `if` statement.
- `st.chat_message("user").write(prompt)`: This displays the user's input in the chat window with the label "user."

**Generate Assistant’s Response**  
```python
with st.chat_message("assistant"):
    st_callback = StreamlitCallbackHandler(st.container())
    response = chain.run(prompt, callbacks=[st_callback])
    st.write(response)
```
- `with st.chat_message("assistant")`: This block specifies that the following content will be displayed as the assistant's message in the chat.
- `st_callback = StreamlitCallbackHandler(st.container())`: A new callback handler is initialized for each interaction to manage streaming and updates for the assistant’s response.
- `response = chain.run(prompt, callbacks=[st_callback])`: This sends the user’s input (`prompt`) to the `chain` (the LangChain agent) for processing. The `callbacks` parameter allows the `StreamlitCallbackHandler` to update the Streamlit app with the response.
- `st.write(response)`: This displays the assistant's generated response in the Streamlit app.

We will create a `app.py` file and can start the app locally from the terminal like this:

`PYTHONPATH=. streamlit run app.py`

**in a nutshell**  
This code creates a simple chat interface in Streamlit where users can enter a prompt. The prompt is processed by a LangChain agent (`chain`), and the response is displayed in the chat window. The `StreamlitCallbackHandler` ensures that the chat interface is updated smoothly as the agent processes the prompt, providing an interactive user experience.